In [2]:
import os
import pandas as pd
from utils import *
from PIL import Image
import copy
from queue import Queue
import threading
import time

In [3]:
path_to_gsv_scrapes = os.path.join("..","dc_scrapes_dump")

root_path = "data"

train_csv_path = "Train.csv"
train_crop_path = os.path.join(root_path, "train")
train_df = pd.read_csv(train_csv_path).sort_values(by="Pano ID")
train_df.head(10)

,Pano ID,SV_x,SV_y,Label,Photographer Heading,Heading,Label ID
237184,--C9ItYhxfNY9d7M4iEdiQ,12400.0,-600.0,1,279.732880,NaN,NaN
89487,--C9ItYhxfNY9d7M4iEdiQ,5400.0,-800.0,8,279.732880,NaN,NaN
46818,--C9ItYhxfNY9d7M4iEdiQ,8000.0,-800.0,8,279.732880,NaN,NaN
132006,--C9ItYhxfNY9d7M4iEdiQ,10500.0,-1000.0,8,279.732880,NaN,NaN
233072,--C9ItYhxfNY9d7M4iEdiQ,12400.0,-700.0,1,279.732880,NaN,NaN
250125,--C9ItYhxfNY9d7M4iEdiQ,2500.0,-500.0,1,279.732880,NaN,NaN
74471,--HQkJe35pVxBmKNw0qZtw,4800.0,-1300.0,8,135.570007,NaN,NaN
249901,--HQkJe35pVxBmKNw0qZtw,3500.0,-600.0,1,135.570007,NaN,NaN
106486,--HQkJe35pVxBmKNw0qZtw,6100.0,-500.0,8,135.570007,NaN,NaN
33846,--HQkJe35pVxBmKNw0qZtw,10400.0,-100.0,8,135.570007,NaN,NaN


In [3]:
def bulk_extract_crops(path_to_crop_csv, destination_dir, path_to_gsv_scrapes,num_threads=4):
    '''
    takes a csv of rows:
    Pano ID, SV_x, SV_y, Label, Photog Heading, Heading, Label ID 
    and get depth-proportioned crops around each features described by each row
    writes each crop to a file in a directory within destination_dir named by that label
    '''
    crop_queue = Queue(0)
    csv_file = open(path_to_crop_csv)
    csv_f = csv.reader(csv_file)
    temp = csv.reader(open(path_to_crop_csv))
    total_crops = sum(1 for line in temp)
    counter = 0
    for row in csv_f:
        # skip header row
        if counter == 0:
            counter += 1
            continue
        crop_queue.put(row)
    
    c_threads = []
    for i in range(0, num_threads):
        c_threads.append(threading.Thread(target=crop_thread, args=(crop_queue,destination_dir,total_crops), daemon=True))
        c_threads[i].start()

    while not crop_queue.empty():
        time.sleep(1)

    for i in range(0, num_threads):
        c_threads[i].join(1)

    print("Finished.")

def crop_thread(to_crop,destination_dir,total_size):
    while not to_crop.empty():
        row = to_crop.get()
        row_num = to_crop.qsize()
        to_crop.task_done()
        pano_id = row[0]

        sv_image_x = float(row[1])
        sv_image_y = float(row[2])
        label_type = row[3]
        photographer_heading = float(row[4])

        destination_basename = '{}crop{},{}'.format(pano_id, sv_image_x, sv_image_y)
        crop_destination = os.path.join(destination_dir, str(label_type), destination_basename)
        if not os.path.exists(crop_destination+".jpg"):
            pano_root = os.path.join(path_to_gsv_scrapes, pano_id[:2], pano_id)
            pano_img_path =  pano_root + ".jpg"
            path_to_depth = pano_root+".depth.txt"
            pano_yaw_deg = 180 - photographer_heading

            # Extract the crop
            if os.path.exists(path_to_depth):
                destination_folder = os.path.join(destination_dir, str(label_type))
                if not os.path.isdir(destination_folder):
                    os.makedirs(destination_folder)

                try:
                    depth = None
                    GSV_IMAGE_WIDTH,GSV_IMAGE_HEIGHT = (None,None)
                    pano_img = None

                    with open(path_to_depth, 'rb') as f:
                        depth = np.loadtxt(f)
                    GSV_IMAGE_WIDTH,GSV_IMAGE_HEIGHT = extract_width_and_height(pano_root+".xml")
                    pano_img = Image.open(pano_img_path)

                    make_single_crop(pano_img, GSV_IMAGE_WIDTH, GSV_IMAGE_HEIGHT, depth, pano_id, sv_image_x, sv_image_y, pano_yaw_deg, crop_destination)
                    print( "Successfully extracted crop to {} {}/{}".format(destination_basename,row_num,total_size))
                except Exception as e:
                    print( "Cropping {} at {},{} failed.".format(pano_id, sv_image_x, sv_image_y) )
                    print(e)
            else:
                print( "Panorama image not found for {} at {}".format(pano_id, pano_img_path) )

In [ ]:
bulk_extract_crops(train_csv_path, train_crop_path, path_to_gsv_scrapes, num_threads=12)

Panorama image not found for unG3ry6SRTp_ZKIjz7SIbg at ..\dc_scrapes_dump\un\unG3ry6SRTp_ZKIjz7SIbg.jpg
Panorama image not found for 5xC5h4A-Zll5PWQKgUxpmA at ..\dc_scrapes_dump\5x\5xC5h4A-Zll5PWQKgUxpmA.jpg
Panorama image not found for F7lSprplkNXICdWt4P-aHQ at ..\dc_scrapes_dump\F7\F7lSprplkNXICdWt4P-aHQ.jpg
Panorama image not found for UFGMWAa9DxyUCXJDIfKeHA at ..\dc_scrapes_dump\UF\UFGMWAa9DxyUCXJDIfKeHA.jpg
Panorama image not found for Eqm0y3THaDahnFj6U3WR7w at ..\dc_scrapes_dump\Eq\Eqm0y3THaDahnFj6U3WR7w.jpg
Panorama image not found for jEdMhNoLgPym0UynCpuCZA at ..\dc_scrapes_dump\jE\jEdMhNoLgPym0UynCpuCZA.jpg
Panorama image not found for 0v5blsj4UYIBgScC6TK7Rg at ..\dc_scrapes_dump\0v\0v5blsj4UYIBgScC6TK7Rg.jpg
Panorama image not found for T-XqDITe64JpIVBPEdu3Wg at ..\dc_scrapes_dump\T-\T-XqDITe64JpIVBPEdu3Wg.jpg
Panorama image not found for _7FztT2kzQ8AvVKh_j1mtQ at ..\dc_scrapes_dump\_7\_7FztT2kzQ8AvVKh_j1mtQ.jpg
Panorama image not found for r_gb6OeOTdKrp8orSM29sg at ..\dc_scr

In [9]:
for label_type in train_df["Label"].unique():
    print(label_type)

Index(['Pano ID', 'SV_x', 'SV_y', 'Label', 'Photographer Heading', 'Heading',
       'Label ID'],
      dtype='object')
1
8
3
2
4
